In [1]:
import os, sys
print(sys.path)

['/home/jdelzio/data-engineering-zoomcamp/week_5', '/home/jdelzio/anaconda3/lib/python311.zip', '/home/jdelzio/anaconda3/lib/python3.11', '/home/jdelzio/anaconda3/lib/python3.11/lib-dynload', '', '/home/jdelzio/anaconda3/lib/python3.11/site-packages']


In [3]:
# (if needed) - manually add spark paths
sys.path.append('/home/jdelzio/spark/spark-3.5.1-bin-hadoop3/python/lib/py4j-0.10.9.7-src.zip')
sys.path.append('/home/jdelzio/spark/spark-3.5.1-bin-hadoop3/python')

In [14]:
# required because env vars not updating in notebook for some reason
os.environ["SPARK_HOME"] = "/home/jdelzio/spark/spark-3.5.1-bin-hadoop3"

In [16]:
import pyspark
from pyspark.sql import SparkSession
pyspark.__version__

'3.5.1'

In [17]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/03 02:35:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
# run command line file download
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2021-01.parquet

In [ ]:
df = spark.read.option("header", "true").parquet("fhvhv_tripdata_2021-01.parquet")

In [ ]:
df.head(5)

In [ ]:
# Can see datatypes are not inferred correctly (with csv's spark may infer all datatypes as string)
df.schema

In [18]:
# read as pandas dataframe
import pandas as pd
pd.__version__

/home/jdelzio/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


'2.2.1'

In [22]:
df_pandas = pd.read_parquet("fhvhv_tripdata_2021-01.parquet")
df_pandas.shape

(11908468, 24)

In [27]:
# Don't need the entire dataset for datatype assignment
df_pandas = df_pandas.head(100)
df_pandas.shape

0      923
1     1382
2      849
3      179
4     1228
      ... 
95     955
96    1562
97    1111
98    2404
99    2059
Name: trip_time, Length: 100, dtype: int64

In [25]:
# Initialize a Spark dataframe with this dataframe - check new schema
spark.createDataFrame(df_pandas).schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('originating_base_num', StringType(), True), StructField('request_datetime', TimestampType(), True), StructField('on_scene_datetime', TimestampType(), True), StructField('pickup_datetime', TimestampType(), True), StructField('dropoff_datetime', TimestampType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('trip_miles', DoubleType(), True), StructField('trip_time', LongType(), True), StructField('base_passenger_fare', DoubleType(), True), StructField('tolls', DoubleType(), True), StructField('bcf', DoubleType(), True), StructField('sales_tax', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True), StructField('airport_fee', DoubleType(), True), StructField('tips', DoubleType(), True), StructField('driver_pay', DoubleType(), True), StructField('shared_request_flag',

# Will need to update the schema for a few columns:
types.StructType([
    types.StructField('hvfhs_license_num', StringType(), True), 
    types.StructField('dispatching_base_num', StringType(), True), 
    types.StructField('originating_base_num', StringType(), True), 
    types.StructField('request_datetime', TimestampType(), True), 
    types.StructField('on_scene_datetime', TimestampType(), True), 
    types.StructField('pickup_datetime', TimestampType(), True), 
    types.StructField('dropoff_datetime', TimestampType(), True), 
    types.StructField('PULocationID', IntegerType(), True), 
    types.StructField('DOLocationID', Integer Type(), True), 
    types.StructField('trip_miles', DoubleType(), True), 
    types.StructField('trip_time', DoubleType(), True), 
    types.StructField('base_passenger_fare', DoubleType(), True), 
    types.StructField('tolls', DoubleType(), True), 
    types.StructField('bcf', DoubleType(), True), 
    types.StructField('sales_tax', DoubleType(), True), 
    types.StructField('congestion_surcharge', DoubleType(), True), 
    types.StructField('airport_fee', DoubleType(), True), 
    types.StructField('tips', DoubleType(), True), 
    types.StructField('driver_pay', DoubleType(), True), 
    types.StructField('shared_request_flag', StringType(), True), 
    types.StructField('shared_match_flag', StringType(), True), 
    types.StructField('access_a_ride_flag', StringType(), True), 
    types.StructField('wav_request_flag', StringType(), True), 
    types.StructField('wav_match_flag', StringType(), True)
])

In [28]:
from pyspark.sql import types

In [41]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True), 
    types.StructField('dispatching_base_num', types.StringType(), True), 
    types.StructField('originating_base_num', types.StringType(), True), 
    types.StructField('request_datetime', types.TimestampType(), True), 
    types.StructField('on_scene_datetime', types.TimestampType(), True), 
    types.StructField('pickup_datetime', types.TimestampType(), True), 
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.LongType(), True),
    types.StructField('DOLocationID', types.LongType(), True), 
    types.StructField('trip_miles', types.DoubleType(), True), 
    types.StructField('trip_time', types.LongType(), True), 
    types.StructField('base_passenger_fare', types.DoubleType(), True), 
    types.StructField('tolls', types.DoubleType(), True), 
    types.StructField('bcf', types.DoubleType(), True), 
    types.StructField('sales_tax', types.DoubleType(), True), 
    types.StructField('congestion_surcharge', types.DoubleType(), True), 
    types.StructField('airport_fee', types.DoubleType(), True), 
    types.StructField('tips', types.DoubleType(), True), 
    types.StructField('driver_pay', types.DoubleType(), True), 
    types.StructField('shared_request_flag', types.StringType(), True), 
    types.StructField('shared_match_flag', types.StringType(), True), 
    types.StructField('access_a_ride_flag', types.StringType(), True), 
    types.StructField('wav_request_flag', types.StringType(), True), 
    types.StructField('wav_match_flag', types.StringType(), True)
])

In [42]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .parquet("fhvhv_tripdata_2021-01.parquet")

In [43]:
df.head(10)

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', originating_base_num='B02682', request_datetime=datetime.datetime(2021, 1, 1, 0, 28, 9), on_scene_datetime=datetime.datetime(2021, 1, 1, 0, 31, 42), pickup_datetime=datetime.datetime(2021, 1, 1, 0, 33, 44), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 49, 7), PULocationID=230, DOLocationID=166, trip_miles=5.26, trip_time=923, base_passenger_fare=22.28, tolls=0.0, bcf=0.67, sales_tax=1.98, congestion_surcharge=2.75, airport_fee=None, tips=0.0, driver_pay=14.99, shared_request_flag='N', shared_match_flag='N', access_a_ride_flag=' ', wav_request_flag='N', wav_match_flag='N'),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', originating_base_num='B02682', request_datetime=datetime.datetime(2021, 1, 1, 0, 45, 56), on_scene_datetime=datetime.datetime(2021, 1, 1, 0, 55, 19), pickup_datetime=datetime.datetime(2021, 1, 1, 0, 55, 19), dropoff_datetime=datetime.datetime(2021, 1, 1, 1, 18, 21), PULocationID=152, DO

In [44]:
# At this point we have one large file. We should partition it into multiple files so that we can parallelize the processing into more than one spark cluster
df.repartition(24) #lazy command - doesn't trigger the partitioning yet

DataFrame[hvfhs_license_num: string, dispatching_base_num: string, originating_base_num: string, request_datetime: timestamp, on_scene_datetime: timestamp, pickup_datetime: timestamp, dropoff_datetime: timestamp, PULocationID: bigint, DOLocationID: bigint, trip_miles: double, trip_time: bigint, base_passenger_fare: double, tolls: double, bcf: double, sales_tax: double, congestion_surcharge: double, airport_fee: double, tips: double, driver_pay: double, shared_request_flag: string, shared_match_flag: string, access_a_ride_flag: string, wav_request_flag: string, wav_match_flag: string]

In [45]:
df.write.parquet('fhvhv/2021/01/')

In [1]:
spark.stop()

NameError: name 'spark' is not defined